In [1]:
# | default_exp configs.prompt_manager

In [2]:
# | export
from typing import Dict, Any, Optional
from pathlib import Path
import json

In [8]:
# | export
class PromptsManager:
    """Enhanced prompts management system"""
    
    def __init__(self):
        self.prompts = self._load_default_prompts()
        self.custom_prompts = {}
    
    def _load_default_prompts(self) -> Dict[str, str]:
        """Load default system prompts"""
        return {
            "system": self._get_default_system_prompt(),
        }

    @staticmethod
    def get_system_context():
        """Generate enhanced system context with detailed environment information"""
        
        # Basic system info
        system_info = {
            "os": platform.system(),
            "os_version": platform.version(),
            "architecture": platform.machine(),
            "python_version": platform.python_version(),
        "current_directory": str(Path.cwd()),
        "home_directory": str(Path.home()),
        }
        
        # Project context detection
        cwd = Path.cwd()
        project_indicators = {
            'git_repo': (cwd / '.git').exists(),
            'python_project': any((cwd / f).exists() for f in [".git",'setup.py', 'pyproject.toml', 'requirements.txt']),
            'nbdev_project': (cwd / 'settings.ini').exists() and (cwd / 'nbs').exists(),
            'node_project': (cwd / 'package.json').exists(),
            'docker_project': any((cwd / f).exists() for f in ['Dockerfile', 'docker-compose.yml']),
        }
        
        system_info['project_context'] = project_indicators
        return system_info
    
    def _get_default_system_prompt(self) -> str:
        """Get the default system prompt"""
        system_context = get_system_context()
        project_types = [k.replace('_project', '') for k, v in system_context['project_context'].items() if v]
        project_context_str = f"Detected: {', '.join(project_types)}" if project_types else "Generic project"
        return f"""You are Buddy, an autonomous AI assistant for software development, running under the `buddy chat` CLI command.

        ## Role & Goal
        - Deliver correct, efficient, and safe solutions for software development tasks.
        - Minimize user effort by autonomously planning and executing tasks.
        - Use Markdown for structured, terminal-friendly output with headers, lists, and code blocks.
        - For simple requests, execute directly without overthinking.
        
        ## Task Execution
        - **Simple Tasks**: Single operation, clear outcome (≤2 tools). Execute directly using provided tools without overthinking.
        - **Moderate/Complex Tasks**: Multi-step or architecture-heavy (e.g., keywords like "build", "create", "automate", "pipeline", "system", "framework", "setup", "develop"; multiple technologies or workflows). Call `task_planner` immediately.
        - **Unclear Requests**: Ask clarifying questions before proceeding.
        - **Escalation**: If complexity emerges mid-execution, escalate to `task_planner`.
        
        ## Tool Usage
        - Tool definitions are provided externally. Follow them for parameter usage.
        - Use `task_planner` for multi-step or complex tasks to create plans.
        - Use `fs_read` (discover mode for listing/fuzzy search; extract mode for content with regex/context) and `fs_write` for file operations.
        - Use `code_interpreter` only for small computations or plots; ask to save code afterward.
        - Use `code_quality` for security/maintainability/performance analysis.
        - Use `doc_generator` for `nbdev`-style documentation.
        - Use `debate_agent` only via `task_planner` for complex decisions.
        - Use `introspect` after multi-step executions (>3 steps) or repeated failures.
        - Use `task_monitor` to track progress.
        - Use `memory_manager` for conversation optimization.
        - Use `execute_bash` as fallback for shell-friendly tasks.
        
        ## File Handling
        - Use `fs_read` (discover mode) to verify file paths recursively from the project root.
        - If a file is not found, ask the user for clarification.
        - For writes, use `fs_write` with a preview in logical directories (e.g., `src/`, `app/`).
        - Inspect project structure before generating code to determine appropriate file locations.
        
        ## Error Handling
        - On tool or external API failure (e.g., rate limits, invalid calls), retry, fallback to `execute_bash`, or suggest a manual CLI alternative.
        - Log errors and run `introspect` for repeated failures to adapt strategy.
        - On complexity discovery: "This turned out more complex than expected — escalating to `task_planner`."
        
        ## Output Formatting
        - Use structured formats (e.g., JSON) for tool calls as per external definitions.
        - Use Markdown for narrative responses: `#`, `##` for headers; ✅/❌ for status; numbered steps for workflows; tables for comparisons; ``` for code/commands.
        - Keep responses terminal-friendly and compatible with `rich` rendering.
        - Justify routing choices (e.g., direct execution vs. planning).
        - Summarize plans with ✅/❌ markers before execution.
        
        ## System Context
        - **OS:** {system_context['os']} {system_context['architecture']}
        - **Python:** {system_context['python_version']}
        - **Directory:** {system_context['current_directory']}
        - **Project:** {project_context_str}
        """


In [9]:
# | export
class AnalyzerPrompts:
    """Centralized prompt templates for AgentTaskAnalyzer"""
    
    @staticmethod
    def complexity_analysis(user_input: str) -> str:
        """Prompt for analyzing task complexity"""
        return f"""
                Analyze the complexity of this user request and classify it.
                
                REQUEST: {user_input}
                
                Consider:
                - Number of components/features required
                - Technical complexity and integration needs
                - Time and resource requirements
                - Dependencies and coordination needed
                
                Respond with JSON only:
                {{
                    "complexity": "simple|moderate|complex",
                    "reasoning": "Brief explanation of why this complexity level was chosen"
                }}
                
                Guidelines:
                - simple: Single component, straightforward implementation, minimal dependencies
                - moderate: Multiple components, some integration, moderate complexity
                - complex: Many components, complex integrations, significant coordination needed
                """

    @staticmethod
    def framework_selection(user_input: str, complexity: str) -> str:
        """Prompt for framework selection debate"""
        return f"""
            You are participating in a framework selection debate. Analyze this request and recommend the best frameworks/libraries.
            
            REQUEST: {user_input}
            COMPLEXITY: {complexity}
            
            Debate the following framework categories and recommend ONE specific choice for each relevant category:
            
            CATEGORIES TO CONSIDER:
            - web_framework: Flask vs FastAPI vs Django
            - database: PostgreSQL vs MySQL vs SQLite vs MongoDB
            - ml_framework: scikit-learn vs TensorFlow vs PyTorch
            - container: Docker vs Podman
            - orchestration: Kubernetes vs Docker Swarm
            - ci_cd: GitHub Actions vs GitLab CI vs Jenkins
            - monitoring: Prometheus vs DataDog vs New Relic
            - message_queue: Redis vs RabbitMQ vs Apache Kafka
            - api_docs: Swagger/OpenAPI vs Postman vs Insomnia
            - testing: pytest vs unittest vs nose2
            
            For each relevant category, provide your recommendation with reasoning.
            
            Respond with JSON only:
            {{
                "web_framework": "fastapi",
                "database": "postgresql", 
                "ml_framework": "scikit-learn",
                "reasoning": {{
                    "web_framework": "FastAPI chosen for automatic API docs and async support",
                    "database": "PostgreSQL for ACID compliance and JSON support",
                    "ml_framework": "scikit-learn for rapid prototyping and proven algorithms"
                }}
            }}
            
            Only include categories that are relevant to the request.
            """

    @staticmethod
    def task_decomposition(user_input: str) -> str:
        """Prompt for breaking down complex tasks"""
        return f"""
            Break down this request into detailed tasks for execution using Buddy tools.
            
            REQUEST: {user_input}
            
            NOTE: Framework selection will be handled by a separate task (T000) using debate_agent.
            Your tasks should reference framework selections from T000 where needed.
            
            AVAILABLE BUDDY TOOLS:
            - fs_read: Read files, list directories, search patterns, find files, grep across files
            - fs_write: Create, edit, modify files with diff preview, insert at specific lines
            - execute_bash: Execute bash commands with working directory control and timeout
            - code_interpreter: Execute Python code with visualization support and result capture
            - code_quality: Analyze code quality, detect issues, suggest improvements
            - doc_generator: Generate documentation for code repositories
            - memory_manager: Manage conversation memory and context
            - introspect: Self-analysis and capability assessment
            - debate_agent: Multi-perspective analysis and decision making
            - todo: Task planning and execution management
            
            Create DESCRIPTIVE tasks with atomic action steps. Each task should:
            1. Have detailed description explaining WHY and HOW
            2. Include atomic "actions" array with sub-steps
            3. Use introspect tool for validation where appropriate
            4. NOT include direct solutions in the JSON
            
            Respond with JSON array only:
            [
                {{
                    "id": "T001",
                    "name": "Descriptive task name explaining the purpose",
                    "description": "Detailed explanation of what this task accomplishes, why it's needed, what challenges it addresses, and how it fits into the overall project. Include context about dependencies and expected outcomes.",
                    "complexity": "simple|moderate|complex",
                    "dependencies": ["T001"],
                    "buddy_tools": ["fs_write", "execute_bash", "introspect"],
                    "frameworks": {{"web_framework": "fastapi", "database": "postgresql"}},
                    "actions": [
                        {{
                            "step": 1,
                            "action": "Analyze current environment and requirements",
                            "tool": "introspect",
                            "purpose": "Understand current capabilities and validate prerequisites",
                            "sub_steps": [
                                "Check system capabilities",
                                "Validate tool availability",
                                "Assess resource requirements"
                            ]
                        }},
                        {{
                            "step": 2,
                            "action": "Create project structure",
                            "tool": "fs_write",
                            "purpose": "Establish organized directory layout",
                            "sub_steps": [
                                "Create main directories",
                                "Set up configuration files",
                                "Initialize project metadata"
                            ]
                        }},
                        {{
                            "step": 3,
                            "action": "Validate setup completion",
                            "tool": "introspect",
                            "purpose": "Verify all components are properly configured",
                            "sub_steps": [
                                "Check directory structure",
                                "Validate file permissions",
                                "Confirm setup integrity"
                            ]
                        }}
                    ],
                    "success_criteria": "Detailed criteria for validating task completion using Buddy tools",
                    "expected_outputs": ["specific_file.py", "config.json"]
                }}
            ]
            
            Requirements:
            - Make tasks VERY descriptive with detailed context
            - Include introspect tool for validation steps
            - Create atomic action steps with sub-steps
            - Focus on WHAT to do, not HOW to implement
            - Use realistic Buddy tool combinations
            - Include frameworks field with relevant tools from the recommended list
            """
